# Pattern Associator

We recreate the network of the [Pattern Associator tutorial from the CECN1 notebook](https://grey.colorado.edu/CompCogNeuro/index.php/CECN1_Pattern_Associator) exploring how the delta rule works and behave. 

In [129]:
%load_ext autoreload
%autoreload 2

In [130]:
import numpy as np
import dotdot
import leabra
import graphs

In [2]:
import random
random.seed(0)

In [315]:
inputsize = 5
outputsize = inputsize
hiddensize = 4*inputsize
numpatterns = 2
numhots = 2


In [138]:
def generate_hot_vec(size, hots):
    retval = np.zeros(size)
    for i in range(hots):
        ix = 0
        while True:
            ix = random.randint (0, size-1)
            if retval[ix] != 1.0:
                break
        retval[ix] = 1.0

    return retval

In [139]:
def is_in_matrix(vec, matrix):
    ''' vec - row vector to test
    matrix - matrix possibly containing row vector
    '''
    for i in range(matrix.shape[0]):
        if np.array_equal(vec, matrix[i]): return True
    return False


In [140]:
def generate_unique_patterns(rows, cols, hots):
    retval = np.zeros((rows, cols))
    for j in range(rows):
        vec = np.zeros(cols)
        while True:
            vec = generate_hot_vec(cols, hots)
            if not is_in_matrix(vec, retval): break
        retval[j] = vec
    return retval

In [316]:
input_pats = generate_unique_patterns(numpatterns, inputsize, numhots)
output_pats = generate_unique_patterns(numpatterns, outputsize, numhots)


In [142]:
u_spec = leabra.UnitSpec(act_thr=0.5, act_gain=100, act_sd=0.01, 
                         g_bar_e=1.0, g_bar_i=1.0, g_bar_l=0.1, 
                         e_rev_e=1.0, e_rev_i=0.25, e_rev_l=0.3,
                         avg_l_min=0.1, avg_l_init=0.4,
                         adapt_on=False)

In [143]:
input_layer  = leabra.Layer(inputsize, unit_spec=u_spec, name='input_layer')
output_spec  = leabra.LayerSpec(g_i=1.5, ff=1.0, fb=0.5, fb_dt=1/1.4, ff0=0.1)
output_layer = leabra.Layer(outputsize, spec=output_spec, unit_spec=u_spec, name='output_layer')
hidden_layer = leabra.Layer(hiddensize, unit_spec=u_spec, name='hidden_layer')

In [309]:
conspec_l = leabra.ConnectionSpec(proj='full', lrule='leabra', lrate=0.04)
conspec = leabra.ConnectionSpec(proj='full', lrule='', lrate=0.04)

inp_hidden_conn = leabra.Connection(input_layer, hidden_layer, spec=conspec) # no learning from input to hidden

hidden_out_conn = leabra.Connection(hidden_layer, output_layer, spec=conspec_l) # learning from hidden to output

out_hidden_conn = leabra.Connection(output_layer, hidden_layer, spec=conspec_l) # learning from output to hidden

inp_out_conn = leabra.Connection(input_layer, output_layer, spec=conspec_l)

In [310]:
# [inp_hidden_conn, hidden_out_conn]
# [inp_out_conn]
network = leabra.Network(layers=[input_layer, hidden_layer, output_layer], connections=[inp_hidden_conn, hidden_out_conn, out_hidden_conn])
network.build()

In [182]:
def event(k, network):
    """Run a minus phase and a plus phase for a given input/output pair"""
    inputs = input_pats[k]
    outputs = output_pats[k]
    # inputs  = np.zeros(inputsize) #[0.0, 0.0, 0.0, 0.0]
    # outputs = np.zeros(outputsize) # [0.0, 0.0]
    # inputs[k] = 1.0
    # outputs[int(k/2)] = 1.0  # desired output

    network.set_inputs({'input_layer': inputs})
    network.set_outputs({'output_layer': outputs})

    # minus phase
    for _ in range(3):
        network.quarter()
        # print('g_e', [u.g_e for u in output_layer.units])
        # print('v_m', [u.v_m for u in output_layer.units])
        # print('v_m_eq', [u.v_m_eq for u in output_layer.units])
        # print('act', output_layer.activities)
        # print(input_layer.activities)
        # print([u.avg_s_eff for u in input_layer.units])
        
    error = sum((np.array(output_layer.activities) - outputs)**2) 

    # plus phase: the output is set directly
    network.quarter()
    
    return error

In [183]:
inp_hidden_conn.weights
print(output_layer.to_connections)

[<leabra.connection.Connection object at 0x7f888eba3190>, <leabra.connection.Connection object at 0x7f8895a1ee50>]


In [148]:
event(0, network)

2.460662913447386

In [150]:
def trial():
    sse = 0.0
    for i in range(numpatterns):
      sse += event(i, network)  
    # sse += event(0, network)
    # sse += event(1, network)
    # sse += event(2, network)
    # sse += event(3, network)
    return sse / numpatterns

In [184]:
err = [trial() for _ in range(200)]

In [ ]:
conn.weights

In [185]:
graphs.line(range(len(err)), err, title="Average error over trials", width=600)

In [179]:
max(err)

1.8053570178876517

In [326]:
ta = np.ones(3)
tb = 2*np.ones(3)
c = zip(ta,tb)
for i in c: print(i)

(1.0, 2.0)
(1.0, 2.0)
(1.0, 2.0)


In [329]:
def hamming_dist(a, b):
    """Count the number of  different ixes"""
    assert len(a)==len(b), 'len a: ' + str(len(a)) + ' != len(b): ' + str(len(b))
    ctr = 0
    for i in zip(a,b):
        if i[0]==0 and i[1] != 0 or i[0] > 0 and i[1] == 0 : ctr+=1
    return ctr


In [333]:
ta = [0.,1.,0.]
tb = [1., 1., 0]
hamming_dist(ta, tb)

1

In [334]:
def event_vec(inputs, outputs, network):
    network.set_inputs({'input_layer': inputs})
    network.set_outputs({'output_layer': outputs})

    # minus phase
    for _ in range(3):
        network.quarter()
        # print('g_e', [u.g_e for u in output_layer.units])
        # print('v_m', [u.v_m for u in output_layer.units])
        # print('v_m_eq', [u.v_m_eq for u in output_layer.units])
        # print('act', output_layer.activities)
        # print(input_layer.activities)
        # print([u.avg_s_eff for u in input_layer.units])
        
    error = sum((np.array(output_layer.activities) - outputs)**2) 
    hd = hamming_dist(output_layer.activities, outputs)
    # print (output_layer.activities)
    out_act = output_layer.activities
    # plus phase: the output is set directly
    network.quarter()
    
    return (error, out_act, hd)

In [335]:
def trial_vec(inputs, outputs):
    sse = 0.0
    act_mat = []
    hd_mn = 0.0
    for i in range(inputs.shape[0]):
        (err, act, hd) = event_vec(inputs[i], outputs[i], network)
        sse += err
        hd_mn += hd
        act_mat.append(act)

    # print()
    return (sse / inputs.shape[0], act_mat, hd_mn / inputs.shape[0])


In [336]:
rows = 2
cols = inputsize
hots = 2
inp = generate_unique_patterns(rows, cols, hots)
outp = generate_unique_patterns(rows, cols, hots)
err_act = [trial_vec(inp, outp) for _ in range(400)]

out_acts = []
err = []
hds = []
for e in err_act:
    out_acts.append(e[1])
    err.append(e[0])
    hds.append(e[2])

o = np.array(out_acts)


In [337]:
outp

array([[1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0.]])

In [340]:
graphs.line(range(len(err)), err, title="Average error over trials", width=600)

In [339]:
graphs.line(range(len(hds)), hds, title="Average hamming dist over trials", width=600)

In [291]:
#palette = ["f7c1bb","885a5a","353a47","84b082","dc136c"]
palette = ["264653","2a9d8f","e9c46a","f4a261","e76f51"]

In [341]:
width = 600
height = 600

for j in range(o.shape[1]):
    figg = graphs.figure(plot_width=width, plot_height=height, tools="")
    for i in range(o.shape[2]):
        graphs.line(range(o.shape[0]), o[:, j, i], fig = figg, show=False, color='#' + palette[i], legend = str(i), line_width=4) #, title="Average error over trials", width=600)
        #graphs.line(range(a.shape[0]), a[:,4], fig = figg)#, title="Average error over trials", width=600)
    graphs.show(figg)

In [160]:
min(err)

0.8981618522441439

In [159]:
err[99]

1.2847766603316746

In [119]:
hidden_out_conn.weights

array([[3.41130565e-02, 1.57647137e-01, 1.48970001e-01, 2.51810340e-01,
        1.41534792e-01, 1.72877315e-02, 9.43520038e-03, 2.53525322e-04,
        4.24118248e-01, 2.08232462e-02],
       [1.55924841e-01, 3.09186870e-02, 7.62438510e-01, 3.11932048e-01,
        2.98783817e-01, 1.65398298e-01, 1.75806040e-02, 5.02979198e-04,
        9.08146452e-01, 7.18911866e-03],
       [9.65108925e-02, 1.99062727e-01, 8.86469139e-01, 2.27116671e-01,
        8.86767398e-02, 3.30862776e-01, 7.25053589e-02, 8.96403567e-04,
        7.94066036e-02, 1.23164026e-01],
       [6.90670204e-01, 4.91417341e-02, 6.83584552e-01, 2.04117383e-01,
        9.34305024e-02, 3.10795339e-01, 9.35530575e-03, 2.44216032e-03,
        5.20647957e-01, 1.20517213e-02],
       [5.29598933e-02, 1.55549019e-01, 1.45859454e-02, 9.52077767e-02,
        4.74429776e-01, 9.36045433e-03, 5.93469695e-01, 2.46545719e-03,
        2.14610252e-01, 6.50811976e-02],
       [3.14079872e-01, 2.06083007e-02, 2.25281460e-01, 5.74629785e-02,
   